In [1]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms
from pyniryo import NiryoRos, Vision  # Make sure to import necessary modules

# Load the pretrained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# Define transformations for the input
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((480, 640)),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define a mapping for colors
colors = {
    21: (255, 0, 0),  # Example for "robot" (blue)
    22: (0, 255, 0),  # Example for "opponent" (green)
}

# Connect to the robot
# WIFI
# ros_instance = NiryoRos("10.10.10.10")
# ETH
ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

while True:
    img_compressed = vision.get_img_compressed()
    camera_info = vision.get_camera_intrinsics()

    img = pyniryo.uncompress_image(img_compressed)
    img_uncompressed = img
    img = pyniryo.undistort_image(img, camera_info.intrinsics, camera_info.distortion)

    # Preprocess the frame for the model
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

    # Run the model and get the output
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0).byte().cpu().numpy()

    # Create a color mask for the segmented output
    segmented_frame = np.zeros_like(img)
    for label, color in colors.items():
        segmented_frame[output_predictions == label] = color

    # Combine the original frame with the segmented output
    combined_frame = cv2.addWeighted(img, 0.5, segmented_frame, 0.5, 0)

    # Display the results
    cv2.imshow('Segmented Frame', combined_frame)
    cv.imshow("stream - raw", img_uncompressed)
    cv.imshow("stream - undistorted", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms

# Load the pretrained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# Define transformations for the input
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((480, 640)),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define a mapping for colors (you can modify as needed)
colors = {
    21: (255, 0, 0),  # Example for "robot" (blue)
    22: (0, 255, 0),  # Example for "opponent" (green)
}

# Capture video from the robot's camera
camera_url = 'http://<robot_ip>:<port>/video'  # Update with your robot's camera URL
cap = cv2.VideoCapture(camera_url)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    input_tensor = preprocess(frame)
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

    # Run the model and get the output
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0).byte().cpu().numpy()

    # Create a color mask for the segmented output
    segmented_frame = np.zeros_like(frame)
    for label, color in colors.items():
        segmented_frame[output_predictions == label] = color

    # Combine the original frame with the segmented output
    combined_frame = cv2.addWeighted(frame, 0.5, segmented_frame, 0.5, 0)

    # Display the results
    cv2.imshow('Segmented Frame', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import pyniryo
from pyniryo2 import Vision, NiryoRos

# Define the positions for marking (for Tic Tac Toe)
positions = {
    "p1": (257, 146),
    "p2": (327, 139),
    "p3": (400, 139),
    "p4": (255,209),
    "p5": (330, 206),
    "p6": (402, 206),
    "p7": (255, 280),
    "p8": (330, 280),
    "p9": (405, 279),
    "I1": (472,151),
    "I2": (480,258)
}

# Initialize the Niryo robot's ROS instance and vision system
ros_instance = NiryoRos("10.10.10.10")
vision = Vision(ros_instance)

while True:
    # Capture the image from the Niryo robot camera
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    # Resize the frame for display and processing
    frame = cv2.resize(img, (640, 480))

    # Example of marking positions (for Tic Tac Toe) on the frame
    for pos, (x, y) in positions.items():
        cv2.putText(frame, pos, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display the camera feed with Tic Tac Toe field positions
    cv2.imshow('Tic Tac Toe Field', frame)

    # Check if 'q' key has been pressed to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup resources
cv2.destroyAllWindows()
